# 04 Observing renewal and churn

In [1]:
import json
import sqlalchemy
import pandas as pd

from pprint import pprint

In [2]:
# Make a sql connection with sqlalchmey
conn_string = "postgresql://postgres-db/churn?user=postgres&password=password" 
engine = sqlalchemy.create_engine(
    conn_string, connect_args={'options': '-csearch_path={}'.format("socialnet7,public")}
)
conn = engine.connect()
# conn.close()

# Query with Pandas, e.g. list all tables
tables = pd.read_sql_query("SELECT * FROM information_schema.tables;", conn)
tables.head(3)

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action
0,churn,socialnet7,active_period,BASE TABLE,None,None,None,None,None,YES,NO,None
1,churn,socialnet7,event_type,BASE TABLE,None,None,None,None,None,YES,NO,None
2,churn,socialnet7,metric_name,BASE TABLE,None,None,None,None,None,YES,NO,None


In [3]:
with open("/app/fightchurn/listings/conf/socialnet7_listings.json", "r") as f: 
    conf = json.loads(f.read())

chap4 = conf["chap4"]

## Dependent Data

In [4]:
subs = pd.read_sql_query("SELECT * FROM subscription ORDER BY account_id, start_date", conn) 
# table schema already set with connection
# subs = pd.read_sql_query("SELECT * FROM socialnet7.subscription", conn)
display(subs.shape)
subs.head()

(54188, 9)

,id,account_id,product,start_date,end_date,mrr,quantity,units,bill_period_months
0,0,1,'socialnet7',2020-01-28,2020-02-28,9.99,None,None,1
1,1,1,'socialnet7',2020-02-28,2020-03-28,9.99,None,None,1
2,2,1,'socialnet7',2020-03-28,2020-04-28,9.99,None,None,1
3,3,1,'socialnet7',2020-04-28,2020-05-28,9.99,None,None,1
4,4,1,'socialnet7',2020-05-28,2020-06-28,9.99,None,None,1


In [5]:
# find accounts that have subscription gaps greater than 0 day
tmp = (
    subs[["account_id", "start_date", "end_date"]]
    .sort_values(["account_id", "start_date"])
    .assign(prev_end_date=lambda d: d.groupby("account_id")["end_date"].shift(1))
)
tmp.query("abs((start_date - prev_end_date).dt.days) > 0", engine="python")
# no rows means all subsciptions starts with the end of previous subscriptions?!
# TODO: try patch the data to have some subscription start after 7 days from last end

,account_id,start_date,end_date,prev_end_date


In [23]:
event = pd.read_sql_query("SELECT * FROM event LIMIT 15000", conn) 
display(event.shape)
event.head()

(15000, 3)

,account_id,event_time,event_type_id
0,20,2020-01-24 09:37:05,2
1,20,2020-01-24 04:47:58,2
2,20,2020-01-24 02:01:31,2
3,20,2020-01-24 06:45:03,2
4,20,2020-01-24 06:58:32,2


In [41]:
metric = pd.read_sql_query("SELECT * FROM metric ORDER BY RANDOM() LIMIT 10000", conn) 
# by default, metric dataset is empty 
metric.head()

,account_id,metric_time,metric_name_id,metric_value
0,4030,2020-04-12,8,96.0
1,738,2020-03-08,7,1.0
2,3170,2020-02-16,8,29.0
3,9210,2020-03-29,5,1.0
4,2624,2020-02-02,1,3.0


In [42]:
tmp = pd.read_sql_query("SELECT metric_name_id, COUNT(*) FROM metric GROUP BY metric_name_id", conn) 
tmp

,metric_name_id,count
0,0,163450
1,1,144501
2,2,160936
3,3,160510
4,4,154133
5,5,39979
6,6,155804
7,7,124663
8,8,158237


### Result Data

In [13]:
tmp = pd.read_sql_query("SELECT * FROM active_period", conn).sort_values(["account_id", "start_date"])
display(tmp.account_id.is_unique)
tmp.head()
# each rows is the churned status of an account

True

,account_id,start_date,churn_date
5401,1,2020-01-28,None
1477,2,2020-01-13,2020-03-13
188,3,2020-01-11,2020-02-11
4902,4,2020-01-29,None
2084,5,2020-01-10,2020-02-10


In [14]:
tmp = pd.read_sql_query("SELECT * FROM socialnet7.observation", conn).sort_values(["account_id", "observation_date"])
tmp.head()
# eacho row is an observation of an account at a observation date

,account_id,observation_date,is_churn
17446,1,2020-02-23,False
30604,1,2020-03-23,False
19536,1,2020-04-23,False
30480,2,2020-03-08,True
10949,4,2020-02-24,False


## Insert Queries

In [24]:
pprint(chap4)

{'defaults': {'%from_yyyy-mm-dd': '2020-02-09',
              '%to_yyyy-mm-dd': '2020-05-10',
              'mode': 'run',
              'type': 'sql'},
 'list1': {'name': 'ongoing_active_periods',
           'params': {'%gap_interval': '7 day'}},
 'list2': {'name': 'churned_periods', 'params': {'%gap_interval': '7 day'}},
 'list3': {'name': 'active_event_weeks', 'params': {}},
 'list4': {'name': 'observation_dates',
           'params': {'%lead_time': '5 day', '%obs_interval': '1 month'}},
 'list5': {'name': 'dataset',
           'params': {'%metric_interval': '7 day', 'mode': 'save'}},
 'list6': {'name': 'current_customers',
           'params': {'%from_yyyy-mm-dd': '2020-05-10',
                      '%metric_interval': '7 day',
                      '%to_yyyy-mm-dd': '2020-05-10',
                      'mode': 'save'}}}


### 4.3 Identifying active periods from subscriptions

- 4.3.3 Finding active periods that are ongoing
- Listing 4.1

#### active periods

- different from "subscriptions"
- active periods _merge_ any multiple subscriptions and ignores short gaps
- each account can only in one active periods at a time

In [12]:
gap_interval = chap4["list1"]["params"]["%gap_interval"]
to_yyyy_mm_dd = chap4["defaults"]["%to_yyyy-mm-dd"]

gap_interval, to_yyyy_mm_dd

('7 day', '2020-05-10')

In [13]:
# listing_4_1_ongoing_active_periods.sql
query = f"""\
WITH RECURSIVE active_period_params AS (
    SELECT
        interval '{gap_interval}' AS allowed_gap,
        '{to_yyyy_mm_dd}'::date AS calc_date
),
active AS (
  -- find all accounts that are currently active
    SELECT
  DISTINCT
        account_id,
        min(start_date) AS start_date
      FROM socialnet7.subscription
INNER JOIN active_period_params AS params
        ON start_date <= params.calc_date
       AND (end_date > params.calc_date OR end_date IS null) -- subscriptions that haven't ended
  GROUP BY account_id

     UNION

  -- find earlier subscriptions that overlapped with or are continuous with
  -- but older than the subscriptions currently found
    SELECT
        subs.account_id,
        subs.start_date
      FROM socialnet7.subscription AS subs
CROSS JOIN active_period_params AS params
INNER JOIN active -- self, i.e. recursive join
        ON subs.account_id = active.account_id
       AND subs.start_date < active.start_date -- subscriptions that starts before THIS subscription
       AND subs.end_date >= (active.start_date - params.allowed_gap)::date -- subscriptions that ends after THIS subscription
)

-- INSERT INTO active_period (account_id, start_date, churn_date)
     SELECT
        account_id,
        min(start_date) AS start_date,
        NULL::date as churn_date
       FROM active
   GROUP BY account_id, churn_date
""" 
res = pd.read_sql_query(query, conn).sort_values(["account_id", "start_date"])
res.head()

,account_id,start_date,churn_date
3259,1,2020-01-28,None
2760,4,2020-01-29,None
3452,7,2020-01-31,None
3622,8,2020-01-04,None
7213,10,2020-01-04,None


In [14]:
query = f"""\
WITH RECURSIVE active_period_params AS (
    SELECT
        interval '{gap_interval}' AS allowed_gap,
        '{to_yyyy_mm_dd}'::date AS calc_date
),
active AS (
  -- find all accounts that are currently active (i.e. date of to_yyyy_mm_dd)
    SELECT
  DISTINCT
        account_id,
        min(start_date) AS start_date
      FROM subscription
INNER JOIN active_period_params AS params
        ON start_date <= params.calc_date
       AND (end_date > params.calc_date OR end_date IS null) 
       -- start before current date AND end after current date (or it's null)
  GROUP BY account_id

     UNION
  -- find earlier subscriptions that overlapped with or are continuous with
  -- but older than the subscriptions currently found
    SELECT
        subs.account_id,
        subs.start_date
      FROM subscription AS subs
CROSS JOIN active_period_params AS params
INNER JOIN active -- self, i.e. recursive join
        ON subs.account_id = active.account_id -- recursive self join
       AND subs.start_date < active.start_date -- subscriptions that are earlier than THIS subscription
       AND subs.end_date >= (active.start_date - params.allowed_gap)::date -- subscriptions that end after THIS subscription
)
SELECT * FROM active
""" 
res = pd.read_sql_query(query, conn).sort_values(["account_id", "start_date"])
res.head()
# each account's subscriptions, their 

,account_id,start_date
37483,1,2020-01-28
29098,1,2020-02-28
19831,1,2020-03-28
9450,1,2020-04-28
30702,4,2020-01-29


In [15]:
gap_interval = pd.Timedelta(days=7)
to_yyyy_mm_dd = pd.Timestamp(chap4["defaults"]["%to_yyyy-mm-dd"])
gap_interval, to_yyyy_mm_dd

(Timedelta('7 days 00:00:00'), Timestamp('2020-05-10 00:00:00'))

In [16]:
# find all subscriptions that 
# - having next subscription started 
# - having subscriptions coninued
# n.b. this doesn't consider the gap between subscriptions
tmp =( 
    subs[["account_id", "start_date", "end_date"]]
    .sort_values(["account_id", "start_date"])
    .assign(
        next_start_date=lambda d: d.groupby("account_id")["start_date"].shift(-1),
        last_end_date=lambda d: d.groupby("account_id")["end_date"].transform(lambda series: series.iloc[-1]) # max())
    )
    .query("next_start_date == next_start_date") # filter out those don't have next subscription
    .query("last_end_date >= @to_yyyy_mm_dd") # filter out those not in "current" context
    .drop(columns=["next_start_date", "last_end_date"])
    
)
tmp.head(5)

,account_id,start_date,end_date
0,1,2020-01-28,2020-02-28
1,1,2020-02-28,2020-03-28
2,1,2020-03-28,2020-04-28
3,1,2020-04-28,2020-05-28
8,4,2020-01-29,2020-02-29


In [17]:
(
    tmp.groupby("account_id", as_index=False)['start_date'].min()
    .assign(end_date=None)
).head()

,account_id,start_date,end_date
0,1,2020-01-28,None
1,4,2020-01-29,None
2,7,2020-01-31,None
3,8,2020-01-04,None
4,10,2020-01-04,None


#### churned periods

- 4.3.4 Finding active periods ending in churn
- Listing 4.2

In [25]:
gap_interval = chap4["list2"]["params"]["%gap_interval"]
from_yyyy_mm_dd = chap4["defaults"]["%from_yyyy-mm-dd"]
to_yyyy_mm_dd = chap4["defaults"]["%to_yyyy-mm-dd"]

gap_interval, from_yyyy_mm_dd, to_yyyy_mm_dd

('7 day', '2020-02-09', '2020-05-10')

In [26]:
# listing_4_2_churned_periods.sql

query = f"""\
WITH RECURSIVE active_period_params AS (
  SELECT INTERVAL '{gap_interval}' AS allowed_gap,
         '{to_yyyy_mm_dd}'::date AS observe_end,
         '{from_yyyy_mm_dd}'::date AS observe_start
),
end_dates AS (
    SELECT 
  DISTINCT 
        account_id, 
        start_date, 
        end_date, 
        (end_date + allowed_gap)::date AS extension_max 
      FROM 
        subscription 
INNER JOIN active_period_params 
        ON end_date BETWEEN observe_start AND observe_end    
), 
resignups AS (
    SELECT 
  DISTINCT 
        e.account_id, 
        e.end_date   
      FROM end_dates AS e 
INNER JOIN subscription AS s 
        ON e.account_id = s.account_id
       AND s.start_date <= e.extension_max
       AND (s.end_date > e.end_date 
        OR s.end_date IS null)      
),
churns AS (
         SELECT 
             e.account_id, 
             e.start_date, 
             e.end_date AS churn_date    
           FROM 
             end_dates AS e 
LEFT OUTER JOIN resignups AS r  ON e.account_id = r.account_id    
            AND e.end_date = r.end_date
          WHERE r.end_date IS null

          UNION

         SELECT 
              s.account_id, 
              s.start_date, 
              e.churn_date    
           FROM subscription AS s 
     CROSS JOIN active_period_params
     INNER JOIN churns e ON s.account_id=e.account_id
            AND s.start_date < e.start_date
            AND s.end_date >= (e.start_date- allowed_gap)::date
) 

-- INSERT INTO active_period (account_id, start_date, churn_date)
    SELECT
        account_id,
        MIN(start_date) AS start_date,
        churn_date
      FROM churns
  GROUP BY account_id, churn_date
"""
res = pd.read_sql_query(query, conn).sort_values(["account_id", "start_date"])
display(res.shape)
res.head()

(2142, 3)

,account_id,start_date,churn_date
1477,2,2020-01-13,2020-03-13
188,3,2020-01-11,2020-02-11
2084,5,2020-01-10,2020-02-10
1239,6,2020-01-21,2020-04-21
1841,9,2020-01-21,2020-02-21


In [21]:
# find accounts that are churned, and their start and churned date
# what are churned accounts? 
# - having subscriptions terminated before the "current" date
# - having gap between subscriptions, but within allowed differences

gap_interval = pd.Timedelta(days=7)
from_yyyy_mm_dd = pd.Timestamp(chap4["defaults"]["%from_yyyy-mm-dd"])
to_yyyy_mm_dd = pd.Timestamp(chap4["defaults"]["%to_yyyy-mm-dd"])
gap_interval, from_yyyy_mm_dd, to_yyyy_mm_dd

(Timedelta('7 days 00:00:00'),
 Timestamp('2020-02-09 00:00:00'),
 Timestamp('2020-05-10 00:00:00'))

In [22]:
churned_subs = (
    subs[["account_id", "start_date", "end_date"]]
    .assign(last_end_date=lambda d: d.groupby("account_id")["end_date"].transform(lambda series: series.max()))
    .query("end_date >= @from_yyyy_mm_dd & last_end_date <= @to_yyyy_mm_dd")
)
churned_subs.head()

,account_id,start_date,end_date,last_end_date
5,2,2020-01-13,2020-02-13,2020-03-13
6,2,2020-02-13,2020-03-13,2020-03-13
7,3,2020-01-11,2020-02-11,2020-02-11
13,5,2020-01-10,2020-02-10,2020-02-10
14,6,2020-01-21,2020-02-21,2020-04-21


In [23]:
churned = (
    churned_subs.groupby("account_id", as_index=False)
    .agg({"start_date": min, "last_end_date": "last"})
    .sort_values(["account_id", "start_date"])
)
churned.head()

,account_id,start_date,last_end_date
0,2,2020-01-13,2020-03-13
1,3,2020-01-11,2020-02-11
2,5,2020-01-10,2020-02-10
3,6,2020-01-21,2020-04-21
4,9,2020-01-21,2020-02-21


In [24]:
churned.shape

(2142, 3)

In [25]:
res.head()

,account_id,start_date,churn_date
1477,2,2020-01-13,2020-03-13
188,3,2020-01-11,2020-02-11
2084,5,2020-01-10,2020-02-10
1239,6,2020-01-21,2020-04-21
1841,9,2020-01-21,2020-02-21


#### insert execution

In [92]:
gap_interval = chap4["list1"]["params"]["%gap_interval"]
from_yyyy_mm_dd = chap4["defaults"]["%from_yyyy-mm-dd"]
to_yyyy_mm_dd = chap4["defaults"]["%to_yyyy-mm-dd"]

gap_interval, to_yyyy_mm_dd

('7 day', '2020-05-10')

In [101]:
# listing_4_1_ongoing_active_periods.sql
query = f"""\
WITH RECURSIVE active_period_params AS (
    SELECT
        interval '{gap_interval}' AS allowed_gap,
        '{to_yyyy_mm_dd}'::date AS calc_date
),
active AS (
  -- find all accounts that are currently active
    SELECT
  DISTINCT
        account_id,
        min(start_date) AS start_date
      FROM socialnet7.subscription
INNER JOIN active_period_params AS params
        ON start_date <= params.calc_date
       AND (end_date > params.calc_date OR end_date IS null) -- subscriptions that haven't ended
  GROUP BY account_id

     UNION

  -- find earlier subscriptions that overlapped with or are continuous with
  -- but older than the subscriptions currently found
    SELECT
        subs.account_id,
        subs.start_date
      FROM socialnet7.subscription AS subs
CROSS JOIN active_period_params AS params
INNER JOIN active -- self, i.e. recursive join
        ON subs.account_id = active.account_id
       AND subs.start_date < active.start_date -- subscriptions that starts before THIS subscription
       AND subs.end_date >= (active.start_date - params.allowed_gap)::date -- subscriptions that ends after THIS subscription
)

INSERT INTO active_period (account_id, start_date, churn_date)
     SELECT
        account_id,
        min(start_date) AS start_date,
        NULL::date as churn_date
       FROM active
   GROUP BY account_id, churn_date
""" 

In [102]:
with engine.begin() as connection:
    result = connection.execute("DELETE FROM active_period")
result

In [103]:
%%time

tmp = pd.read_sql_query("SELECT * FROM active_period", conn) 
assert tmp.shape[0] == 0, "Table 'active_period' already populated with data"

with engine.begin() as connection:
    result = connection.execute(query)
result

CPU times: user 5.95 ms, sys: 89 µs, total: 6.04 ms
Wall time: 436 ms


In [105]:
# listing_4_2_churned_periods.sql

query = f"""\
WITH RECURSIVE active_period_params AS (
  SELECT INTERVAL '{gap_interval}' AS allowed_gap,
         '{to_yyyy_mm_dd}'::date AS observe_end,
         '{from_yyyy_mm_dd}'::date AS observe_start
),
end_dates AS (
    SELECT 
  DISTINCT 
        account_id, 
        start_date, 
        end_date, 
        (end_date + allowed_gap)::date AS extension_max 
      FROM 
        subscription 
INNER JOIN active_period_params 
        ON end_date BETWEEN observe_start AND observe_end    
), 
resignups AS (
    SELECT 
  DISTINCT 
        e.account_id, 
        e.end_date   
      FROM end_dates AS e 
INNER JOIN subscription AS s 
        ON e.account_id = s.account_id
       AND s.start_date <= e.extension_max
       AND (s.end_date > e.end_date 
        OR s.end_date IS null)      
),
churns AS (
         SELECT 
             e.account_id, 
             e.start_date, 
             e.end_date AS churn_date    
           FROM 
             end_dates AS e 
LEFT OUTER JOIN resignups AS r  ON e.account_id = r.account_id    
            AND e.end_date = r.end_date
          WHERE r.end_date IS null

          UNION

         SELECT 
              s.account_id, 
              s.start_date, 
              e.churn_date    
           FROM subscription AS s 
     CROSS JOIN active_period_params
     INNER JOIN churns e ON s.account_id=e.account_id
            AND s.start_date < e.start_date
            AND s.end_date >= (e.start_date- allowed_gap)::date
) 

INSERT INTO active_period (account_id, start_date, churn_date)
    SELECT
        account_id,
        MIN(start_date) AS start_date,
        churn_date
      FROM churns
  GROUP BY account_id, churn_date
"""

In [107]:
%%time

with engine.begin() as connection:
    result = connection.execute(query)
result

CPU times: user 3.53 ms, sys: 166 µs, total: 3.7 ms
Wall time: 578 ms


### 4.4 Identifying active periods from nonsubscription products

- i.e. from "events" instead of "subscriptions"
- Listing 4.3

In [23]:
from_yyyy_mm_dd = chap4["defaults"]["%from_yyyy-mm-dd"]
to_yyyy_mm_dd = chap4["defaults"]["%to_yyyy-mm-dd"]

from_yyyy_mm_dd, to_yyyy_mm_dd

('2020-02-09', '2020-05-10')

In [24]:
%%time 

query = f"""\
WITH periods AS (
    -- intervals with 7 day differences
    SELECT
        i::timestamp AS period_start,
        i::timestamp + '7 day'::interval AS period_end
      FROM
        generate_series('{from_yyyy_mm_dd}', '{to_yyyy_mm_dd}', '7 day'::interval) AS i
)

-- INSERT INTO active_week (account_id, start_date, end_date)
    SELECT
        account_id,
        period_start::date,
        period_end::date
      FROM
        event
INNER JOIN periods
        ON event_time >= period_start
       AND event_time < period_end
  GROUP BY account_id, period_start, period_end
"""

res = pd.read_sql_query(query, conn)
res.head()

CPU times: user 662 ms, sys: 14.3 ms, total: 677 ms
Wall time: 17.9 s


,account_id,period_start,period_end
0,3622,2020-02-16,2020-02-23
1,4504,2020-02-09,2020-02-16
2,10507,2020-02-09,2020-02-16
3,2604,2020-02-23,2020-03-01
4,6583,2020-02-23,2020-03-01


In [25]:
res[res.account_id == 2049].sort_values("period_start").tail()

,account_id,period_start,period_end
92314,2049,2020-04-12,2020-04-19
101810,2049,2020-04-19,2020-04-26
94774,2049,2020-04-26,2020-05-03
114026,2049,2020-05-03,2020-05-10
91324,2049,2020-05-10,2020-05-17


- active weeks for the account "2049"
- between the given period start and end, the account is active (at least one event)

In [21]:
event.head()

,account_id,event_time,event_type_id
0,13,2020-05-25 15:36:02,3
1,13,2020-05-26 00:37:39,0
2,13,2020-05-26 21:27:55,0
3,13,2020-05-26 07:37:34,0
4,13,2020-05-26 10:32:56,0


In [27]:
query = """SELECT * FROM event WHERE account_id = 2049"""
tmp = pd.read_sql_query(query, conn)
tmp.sort_values("event_time").query("event_time >= '2020-05-10' and event_time <= '2020-05-17'")

,account_id,event_time,event_type_id
106,2049,2020-05-10 19:50:58,2
107,2049,2020-05-12 08:31:58,0
109,2049,2020-05-15 02:40:32,6
108,2049,2020-05-15 07:57:44,3


- the idea is "active_week" is comprable to "subscriptions" table
- i.e. the aggration of events by week can be looked as a subscription of 7 day period
- and if the account is still active in the next week, it's a continuous subscription

In [26]:
res.head()

,account_id,period_start,period_end
0,3622,2020-02-16,2020-02-23
1,4504,2020-02-09,2020-02-16
2,10507,2020-02-09,2020-02-16
3,2604,2020-02-23,2020-03-01
4,6583,2020-02-23,2020-03-01


In [29]:
from_yyyy_mm_dd = pd.Timestamp(chap4["defaults"]["%from_yyyy-mm-dd"])
to_yyyy_mm_dd = pd.Timestamp(chap4["defaults"]["%to_yyyy-mm-dd"])

from_yyyy_mm_dd, to_yyyy_mm_dd

(Timestamp('2020-02-09 00:00:00'), Timestamp('2020-05-10 00:00:00'))

In [30]:
churned_periods = (
   res.assign(last_period_end=lambda d: d.groupby("account_id")["period_end"].transform(lambda series: series.max())) 
    .query("period_end >= @from_yyyy_mm_dd & last_period_end <= @to_yyyy_mm_dd")
)
churned_periods.head()

,account_id,period_start,period_end,last_period_end
8,2929,2020-02-16,2020-02-23,2020-03-22
16,2563,2020-02-16,2020-02-23,2020-04-12
28,1882,2020-02-16,2020-02-23,2020-04-26
32,10123,2020-02-16,2020-02-23,2020-04-19
34,5171,2020-02-09,2020-02-16,2020-04-19


In [32]:
churned = (
    churned_periods.groupby("account_id", as_index=False)
    .agg({"period_start": min, "last_period_end": "last"})
    .sort_values(["account_id", "period_start"])
)
churned.head()

,account_id,period_start,last_period_end
0,2,2020-02-09,2020-03-15
1,3,2020-02-09,2020-02-16
2,5,2020-02-09,2020-02-16
3,6,2020-02-09,2020-04-26
4,9,2020-02-09,2020-02-23


### 4.5 Picking observation Dates

- Listing 4.4
- 4.5.3 Observation data SQL program

In [110]:
active = pd.read_sql_query("SELECT * FROM active_period", conn).sort_values(["account_id", "start_date"])
active.head()
# active_period should have churned or nott status for every account

,account_id,start_date,churn_date
5401,1,2020-01-28,None
1477,2,2020-01-13,2020-03-13
188,3,2020-01-11,2020-02-11
4902,4,2020-01-29,None
2084,5,2020-01-10,2020-02-10


In [111]:
# {'%lead_time': '5 day', '%obs_interval': '1 month'
lead_time = chap4["list4"]["params"]["%lead_time"]
obs_interval = chap4["list4"]["params"]["%obs_interval"]
from_yyyy_mm_dd = chap4["defaults"]["%from_yyyy-mm-dd"]
to_yyyy_mm_dd = chap4["defaults"]["%to_yyyy-mm-dd"]

obs_interval, lead_time, from_yyyy_mm_dd, to_yyyy_mm_dd

('1 month', '5 day', '2020-02-09', '2020-05-10')

In [112]:
# listing_4_4_observation_dates.sql
query = f"""\

WITH RECURSIVE observation_params AS (
  SELECT interval '{obs_interval}' AS obs_interval,
         interval '{lead_time}'  AS lead_time,
         '{from_yyyy_mm_dd}'::date AS obs_start,
         '{to_yyyy_mm_dd}'::date AS obs_end
),
observations AS (
    -- initial observation for accounts
    SELECT
        account_id,
        start_date,
        1 AS obs_count,
        (start_date + obs_interval-lead_time)::date AS obs_date,
      CASE
        WHEN churn_date >= (start_date + obs_interval-lead_time)::date
            AND churn_date < (start_date + 2*obs_interval-lead_time)::date
          THEN true
          ELSE false
         END AS is_churn
      FROM active_period
INNER JOIN observation_params
        ON (churn_date > (obs_start+obs_interval-lead_time)::date
           OR churn_date is null)

  UNION

    SELECT
        obs.account_id,
        obs.start_date,
        obs_count+1 AS obs_count,
        (obs.start_date+(obs_count+1)*obs_interval-lead_time)::date AS obs_date,
      CASE
        WHEN churn_date >= (obs.start_date + (obs_count+1)*obs_interval-lead_time)::date
            AND churn_date < (obs.start_date + (obs_count+2)*obs_interval-lead_time)::date
          THEN true
          ELSE false
       END AS is_churn
      FROM observations AS obs  -- self recursive join
INNER JOIN observation_params
        ON (obs.start_date+(obs_count+1)*obs_interval-lead_time)::date <= obs_end
INNER JOIN active_period AS active
        ON active.account_id = obs.account_id
          AND (obs.start_date+(obs_count+1)*obs_interval-lead_time)::date >= active.start_date
          AND ((obs.start_date+(obs_count+1)*obs_interval-lead_time)::date < active.churn_date 
                OR churn_date IS null)
)

SELECT DISTINCT * FROM observations 

--     SELECT 
--   DISTINCT account_id, obs_date, is_churn
--       FROM observations
-- INNER JOIN observation_params 
--         ON obs_date 
--    BETWEEN obs_start and obs_end
"""

res = pd.read_sql_query(query, conn).sort_values(["account_id", "obs_date"])
res.head(6)

,account_id,start_date,obs_count,obs_date,is_churn
2261,1,2020-01-28,1,2020-02-23,False
17872,1,2020-01-28,2,2020-03-23,False
10310,1,2020-01-28,3,2020-04-23,False
5877,2,2020-01-13,1,2020-02-08,False
19798,2,2020-01-13,2,2020-03-08,True
28376,4,2020-01-29,1,2020-02-24,False


Notes on churned or not observation

- each row is an observation on an account id at a given time interval, i.e. take snapshots of accounts when they don't churn
- observe every account on the same periodic cycle that you use when calculating the churn rate
- note that observation is has a lead time, which tries to observe user's behaviour before the end of a subscription period
- account 1 has 3 observations in the given observation period, with one month interval
  - the user isn't churned, each observation is taken 5 days before next subscription starts
  - ...
- account 2 has 2 observations
  - first obsevation is taken 5 days before the subscription ends (2020/02/13 - 5 day)
  - second observation is taken 5 days before the subscription ends, and this time it ends in churn

In [ ]:
df = res.copy()

#### insert execution

In [32]:
obs_interval = chap4["list4"]["params"]["%obs_interval"]
lead_time = chap4["list4"]["params"]["%lead_time"]
from_yyyy_mm_dd = chap4["defaults"]["%from_yyyy-mm-dd"]
to_yyyy_mm_dd = chap4["defaults"]["%to_yyyy-mm-dd"]

obs_interval, lead_time, from_yyyy_mm_dd, to_yyyy_mm_dd

('1 month', '5 day', '2020-02-09', '2020-05-10')

In [33]:
# listing_4_4_observation_dates.sql
# insert into "observation" table
query = f"""\

WITH RECURSIVE observation_params AS (
  SELECT interval '{obs_interval}' AS obs_interval,
         interval '{lead_time}'  AS lead_time,
         '{from_yyyy_mm_dd}'::date AS obs_start,
         '{to_yyyy_mm_dd}'::date AS obs_end
),
observations AS (
    SELECT
        account_id,
        start_date,
        1 AS obs_count,
        (start_date + obs_interval-lead_time)::date AS obs_date,
      CASE
        WHEN churn_date >= (start_date + obs_interval-lead_time)::date
            AND churn_date <  (start_date + 2*obs_interval-lead_time)::date
          THEN true
          ELSE false
         END AS is_churn
      FROM active_period
INNER JOIN observation_params
        ON (churn_date > (obs_start+obs_interval-lead_time)::date
           OR churn_date is null)

  UNION

    SELECT
        obs.account_id,
        obs.start_date,
        obs_count+1 AS obs_count,
        (obs.start_date+(obs_count+1)*obs_interval-lead_time)::date AS obs_date,
      CASE
        WHEN churn_date >= (obs.start_date + (obs_count+1)*obs_interval-lead_time)::date
            AND churn_date < (obs.start_date + (obs_count+2)*obs_interval-lead_time)::date
          THEN true
          ELSE false
       END AS is_churn
      FROM observations AS obs  -- self recursive join
INNER JOIN observation_params
        ON (obs.start_date+(obs_count+1)*obs_interval-lead_time)::date <= obs_end
INNER JOIN active_period AS active
        ON active.account_id = obs.account_id
          AND (obs.start_date+(obs_count+1)*obs_interval-lead_time)::date >= active.start_date
          AND ((obs.start_date+(obs_count+1)*obs_interval-lead_time)::date < active.churn_date 
                OR churn_date IS null)
)

INSERT INTO socialnet7.observation (account_id, observation_date, is_churn)
    SELECT 
  DISTINCT account_id, obs_date, is_churn
      FROM observations
INNER JOIN observation_params 
        ON obs_date 
   BETWEEN obs_start and obs_end
"""

In [34]:
with engine.begin() as connection:
    result = connection.execute("DELETE FROM observation")
result

In [35]:
tmp = pd.read_sql_query("SELECT * FROM observation", conn) 
assert tmp.shape[0] == 0, "Table 'observation' already populated with data"

with engine.begin() as connection:
    result = connection.execute(query)
result

### 4.6 Exporting a churn dataset

- Listing 4.5
- 4.6.1 Dataset creation SQL program

In [43]:
metric_interval = chap4["list6"]["params"]["%metric_interval"]
from_yyyy_mm_dd = chap4["defaults"]["%from_yyyy-mm-dd"]
to_yyyy_mm_dd = chap4["defaults"]["%to_yyyy-mm-dd"]

metric_interval, from_yyyy_mm_dd, to_yyyy_mm_dd

('7 day', '2020-02-09', '2020-05-10')

In [44]:
query = f"""\

WITH observation_params AS (
    SELECT  
        interval '{metric_interval}' AS metric_period,
        '{from_yyyy_mm_dd}'::timestamp AS obs_start,
        '{to_yyyy_mm_dd}'::timestamp AS obs_end
)

    SELECT 
        metric.account_id, 
        obs.observation_date, 
        obs.is_churn,
        SUM(CASE WHEN metric_name_id=0 THEN metric_value ELSE 0 END) AS like_per_month,
        SUM(CASE WHEN metric_name_id=1 THEN metric_value ELSE 0 END) AS newfriend_per_month,
        SUM(CASE WHEN metric_name_id=2 THEN metric_value ELSE 0 END) AS post_per_month,
        SUM(CASE WHEN metric_name_id=3 THEN metric_value ELSE 0 END) AS adview_per_month,
        SUM(CASE WHEN metric_name_id=4 THEN metric_value ELSE 0 END) AS dislike_per_month,
        SUM(CASE WHEN metric_name_id=5 THEN metric_value ELSE 0 END) AS unfriend_per_month,
        SUM(CASE WHEN metric_name_id=6 THEN metric_value ELSE 0 END) AS message_per_month,
        SUM(CASE WHEN metric_name_id=7 THEN metric_value ELSE 0 END) AS reply_per_month,
        SUM(CASE WHEN metric_name_id=8 THEN metric_value ELSE 0 END) AS account_tenure
      FROM metric
INNER JOIN observation_params AS params
        ON metric_time BETWEEN params.obs_start AND params.obs_end    
INNER JOIN observation AS obs
        ON metric.account_id = obs.account_id
       AND metric.metric_time > (obs.observation_date - params.metric_period)::timestamp    
       AND metric.metric_time <= obs.observation_date::timestamp
  GROUP BY metric.account_id, metric.metric_time, obs.observation_date, obs.is_churn    
  ORDER BY obs.observation_date, metric.account_id
"""

res = pd.read_sql_query(query, conn).sort_values(["account_id", "observation_date"])
res.head(6)

,account_id,observation_date,is_churn,like_per_month,newfriend_per_month,post_per_month,adview_per_month,dislike_per_month,unfriend_per_month,message_per_month,reply_per_month,account_tenure
4112,1,2020-02-23,False,2.0,0.0,6.0,1.0,3.0,1.0,6.0,6.0,26.0
14104,1,2020-03-23,False,2.0,1.0,6.0,2.0,2.0,2.0,7.0,4.0,54.0
24621,1,2020-04-23,False,1.0,1.0,2.0,4.0,4.0,1.0,7.0,3.0,82.0
9178,2,2020-03-08,True,67.0,7.0,49.0,124.0,120.0,0.0,0.0,0.0,55.0
4398,4,2020-02-24,False,36.0,4.0,27.0,36.0,5.0,1.0,32.0,8.0,25.0
14425,4,2020-03-24,False,37.0,8.0,39.0,40.0,9.0,0.0,39.0,3.0,53.0


About churn dataset

- each row is an observation, and the observation is from previous "active_period" calculations
- account 1 has 3 observations in the given period 
- accoutn 2 has churned, only 1 observation before the churned period is kept